In [20]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pickle
nltk.download('stopwords')
nltk.download('punkt')
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aivendil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aivendil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Препроцессинг текста
Удаляем html теги, пунктуацию, приводим слова к инфнитиву и разбиваем строку по пробелам

In [21]:
def preprocess_text(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    mystem = Mystem()
    russian_stopwords = stopwords.words("russian")
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return tokens

In [33]:
def drop_tags(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    mystem = Mystem()
    russian_stopwords = stopwords.words("russian")
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

# Загрузка датасета

In [28]:
dataset = pd.read_csv('extractvacancy_vacancy_t.csv')
dataset.head()
dataset.fillna('', inplace=True)
dataset.shape

(5525, 10)

# Формирование корпуса, на котором будет производится обучение.

In [ ]:
dataset['full_sting'] = '{name} {duty} {qualification}'.format(name=dataset.name, duty=dataset.duty, qualification=dataset.qualification)
# dataset.name + dataset.duty
dataset.full_sting= dataset.full_sting.map(drop_tags)

# Обучение векторизатора

In [13]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(dataset.full_sting)    

In [14]:
pickle.dump(vectorizer, open('vectorizer_100', 'wb'))


In [15]:
kmeans = KMeans(n_clusters=10)
pred = kmeans.fit_predict(X)

/home/aivendil/.local/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return_n_iter=True)


In [19]:
pred = pd.Series(pred)
pickle.dump(kmeans, open('kmeans_100','wb'))